In [3]:
import pandas as pd
import numpy as np
import random as rnd

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ .load the `txt` file of the previous point and convert it to a `csv` file by hand

In [68]:
#creo una lista di numeri interi
numbers=list(range(1,10+1))
#print(f"List: {numbers}")

int_file_name = "data/data_int.txt" #nome del file txt
with open(int_file_name, 'w') as txt_data: #apro il file (ogni volta sovrascivo i valori se presenti)
    for i in numbers:
        #inserisco un numero per ogni riga
        txt_data.write(str(i)+' ') # write last line to a file

!cat data/data_int.txt

#creo una lista di 5x5 float
matrix_list=np.arange(1.,26.,1.)

#per controllare creo la matrice e a stampo
#matrix=matrix_list.reshape((5,5))
#print(f"\n\nMatrix:\n{matrix}")
print('\n')

float_file_name = "data/data_float.txt" #nome del file txt
with open(float_file_name, 'w') as txt_data: #apro il file (ogni volta sovrascivo i valori se presenti)

    for index, item in enumerate(matrix_list):
        if(item<10):
            if(index%5==0 and index>1): #ogni 5 valori vado a capo (matrice 5x5)
                txt_data.write('\n'+str(item)+'   ')
            else:
                txt_data.write(str(item)+'   ')
        else:
            if(index%5==0 and index>1): #ogni 5 valori vado a capo (matrice 5x5)
                txt_data.write('\n'+str(item)+'  ')
            else:
                txt_data.write(str(item)+'  ')

!cat data/data_float.txt

print('\n')
#conversione automatica
read_file = pd.read_csv(float_file_name)
read_file.to_csv (r'data/data_float.csv', index=None)

#conversione manuale
txt_lines = open(float_file_name, "r").readlines() #apro il file txt e leggo le linee
floatCsv_file_name = "data/data_float.csv" #nome del file csv
with open(floatCsv_file_name, 'w') as csv_data: #apro il file (ogni volta sovrascivo i valori se presenti)
    for i in txt_lines:
        #inserisco tutte le righe del file txt
        csv_data.write(str(i)) # write last line to a file

!cat data/data_float.csv

1 2 3 4 5 6 7 8 9 10 

1.0   2.0   3.0   4.0   5.0   
6.0   7.0   8.0   9.0   10.0  
11.0  12.0  13.0  14.0  15.0  
16.0  17.0  18.0  19.0  20.0  
21.0  22.0  23.0  24.0  25.0  

1.0   2.0   3.0   4.0   5.0   
6.0   7.0   8.0   9.0   10.0  
11.0  12.0  13.0  14.0  15.0  
16.0  17.0  18.0  19.0  20.0  
21.0  22.0  23.0  24.0  25.0  

2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [80]:
import urllib.request
url = 'https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json'
res=[]
with urllib.request.urlopen(url) as json_data:
    #print(data_file.read(300))
    for line in json_data:
        output_dict = [x for x in line if x['CreditCardType'] == 'American Express']
        res.append(output_dict)

KeyboardInterrupt: 

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?